In [16]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
# MacOS上画图中文乱码的问题
# https://blog.csdn.net/minixuezhen/article/details/81516949?utm_medium=distribute.pc_relevant.none-task-blog-
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

%matplotlib inline

import sys
import os

from pprint import pprint

In [84]:
PLACES = ['A', 'B', 'C', 'A1', 'A2', 'A3']
TYPES = [0, 1, 2]
def _load_data():
    table_files = """data/附件1 监测点A空气质量预报基础数据.xlsx
data/附件2 监测点B、C空气质量预报基础数据.xlsx
data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx""".split("\n")
    tables = [pd.read_excel("../"+i, engine='openpyxl', sheet_name=None) for i in table_files] # 表格全读取
    keys = [list(i.keys()) for i in tables] # 每个表格的sheet list
    return tables, keys

def _process_to_json(tables):
    sheetnames = {0:'监测点{}逐小时污染物浓度与气象一次预报数据', 1:'监测点{}逐小时污染物浓度与气象实测数据', 2:'监测点{}逐日污染物浓度实测数据'}
    result = {i:{} for i in PLACES}
    
    for k,v in sheetnames.items():
        tables[0][v.format('A')].name = v.format('A')
        result['A'][k] = tables[0][v.format('A')]
    for c in ['B', 'C']:
        for k,v in sheetnames.items():
            tables[1][v.format(c)].name = v.format(c)
            result[c][k] = tables[1][v.format(c)]
    for c in ['A1', 'A2', 'A3']:
        for k,v in sheetnames.items():
            tables[2][v.format(c)].name = v.format(c)
            result[c][k] = tables[2][v.format(c)]
    return result

def load(): 
    tables, keys = _load_data()
    return _process_to_json(tables)

data = load()

In [87]:
print(data.keys())
print(data['A'].keys())
# for p in PLACES:
#     for i in range(3): 
#         print(data[p][i].info())

dict_keys(['A', 'B', 'C', 'A1', 'A2', 'A3'])
dict_keys([0, 1, 2])


In [167]:
def null_values_analysis(data):
    NULL_COUNTS = {}
    for p in PLACES:
        for t in TYPES:  
            if data[p][t].isnull().sum().sum():
                NULL_COUNTS.update({p+"_"+str(t):data[p][t].isnull().sum()})
    return NULL_COUNTS

def null_values_analysis_plot(null_table):
    fig, axs = plt.subplots(3, 3, figsize=(25,16))
    fig.tight_layout(h_pad=10)
    for i, (k,v) in enumerate(null_table.items()):
        axs[i//3][i%3].bar(v.keys(), v, 0.35)
        axs[i//3][i%3].set_title(k)
        axs[i//3][i%3].set_xticklabels(labels=v.keys(),rotation=45)
        axs[i//3][i%3].grid(True)

    font = {'family': 'serif',
            'color':  'darkred',
            'weight': 'normal',
            'size': 16,
            }
    fig.suptitle('空值分析', size=20)
    plt.savefig("results/nan_value_analysis.jpg")
    
null_table = (null_values_analysis(data))
# null_values_analysis_plot(null_table)